In [25]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
import os

In [2]:
%pwd

'/Users/harendrakumar/Documents/Demand_forecast/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/harendrakumar/Documents/Demand_forecast'

In [14]:
df = pd.read_csv("artifacts/data_transformation/train.csv")
df.index

RangeIndex(start=0, stop=46, step=1)

In [15]:
df['Date'] = pd.to_datetime(df['Date'])

In [8]:
df = df[['Date', 'Product_ID', 'Demand', 'Inventory']]

In [20]:
df = df['Demand']

In [22]:
df.index[-1]

Timestamp('2023-06-03 00:00:00')

In [30]:
model = SARIMAX(df, order= (1, 1, 1), seasonal_order= (1, 1, 1, 2))
model_fit = model.fit(disp=False)
predictions = model_fit.predict(len(df), len(df) + 10 -1)
predictions = predictions.astype(int)
predictions

/Users/harendrakumar/.local/share/virtualenvs/ETL-6mpoCg48/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/harendrakumar/.local/share/virtualenvs/ETL-6mpoCg48/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/harendrakumar/.local/share/virtualenvs/ETL-6mpoCg48/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/harendrakumar/.local/share/virtualenvs/ETL-6mpoCg48/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:471: Va

46    149
47    113
48    130
49    129
50    136
51    126
52    135
53    128
54    137
55    128
Name: predicted_mean, dtype: int64

In [31]:
future_dates = pd.date_range(start=df.index[-1] + pd.DateOffset(1), periods=10, freq='D')
future_dates

DatetimeIndex(['2023-06-04', '2023-06-05', '2023-06-06', '2023-06-07',
               '2023-06-08', '2023-06-09', '2023-06-10', '2023-06-11',
               '2023-06-12', '2023-06-13'],
              dtype='datetime64[ns]', freq='D')

In [70]:
predictions = predictions.to_list()
pd.Series(data=predictions, index=future_dates)

2023-06-04    149
2023-06-05    113
2023-06-06    130
2023-06-07    129
2023-06-08    136
2023-06-09    126
2023-06-10    135
2023-06-11    128
2023-06-12    137
2023-06-13    128
Freq: D, dtype: int64

In [54]:
forecasted_demand = pd.DataFrame(data=predictions)
forecasted_demand.reset_index(inplace=True)
forecasted_demand.set_index(future_dates, inplace=True)
forecasted_demand = forecasted_demand['predicted_mean']
forecasted_demand

2023-06-04    149
2023-06-05    113
2023-06-06    130
2023-06-07    129
2023-06-08    136
2023-06-09    126
2023-06-10    135
2023-06-11    128
2023-06-12    137
2023-06-13    128
Freq: D, Name: predicted_mean, dtype: int64